In [ ]:
import pandas as pd
import csv
import datetime
import yfinance as yf
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup as bs
import requests
from pyspark.sql.functions import sum,max,min,mean,count
import datetime as dt
import pyspark
from pyspark.sql import SparkSession
import findspark
import yaml
from yaml.loader import SafeLoader
from os.path import abspath

warehouse_location = abspath('spark-warehouse')
with open('cfg.yml') as f:
    config = yaml.load(f, Loader = SafeLoader)

    #create spark connection
findspark.init()
spark = SparkSession.builder \
    .master(config['spark']['spark_master'])\
    .appName('retrieve')\
    .enableHiveSupport()\
    .config('spark.sql.warehouse.dir', warehouse_location)\
    .config(config['spark']['spark_jars'], config['spark']['spark_jars_path'])\
    .config('spark.cores.max', '2')\
    .config('spark.executor.cores', '2')\
    .getOrCreate()
spark.sparkContext.setLogLevel("WARN")
spark

#create database config details
url = config['postgres']['url']
properties = {
    'user': config['postgres']['user'],
    'password' : config['postgres']['user'],
    'url': url,
    'driver': config['postgres']['driver']
    
}

23/04/03 20:18:39 WARN Utils: Your hostname, cis6180 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/04/03 20:18:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/04/03 20:18:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Retrieve Data from Database and Write to Csv

In [ ]:
import pyspark.pandas as ps

#retrieve data from database
def return_data(ticker_list, from_date, to_date):
    sentiment = spark.read.format("jdbc")\
        .option("url", "jdbc:postgresql://localhost:5432/financials") \
        .option("driver", "org.postgresql.Driver").option("dbtable", "sentiment") \
        .option("user", "adam").option("password", "green").load()
    finance = spark.read.format("jdbc")\
        .option("url", "jdbc:postgresql://localhost:5432/financials") \
        .option("driver", "org.postgresql.Driver").option("dbtable", "company_data") \
        .option("user", "adam").option("password", "green").load()
    full_data = finance.join(sentiment, ['date', 'ticker'], 'leftouter').fillna(0)
    df_list = []
    for ticker in ticker_list:
        working_data = full_data[full_data['ticker'] == ticker]
        print(working_data.count())
        working_data = working_data.sort('date', ascending = True).filter((working_data.date >= from_date) & (working_data.date<= to_date)).toPandas().set_index('date')
        #sorted_data = working_data.toPandas().loc[from_date:to_date,]
        print(ticker, working_data.shape)
        working_data = working_data[~working_data.index.duplicated()]
        print(working_data)
        working_data.to_csv('data/'+ticker+'_dataframe.csv')
        df_list.append(working_data)
return_data(['MSFT', 'GOOG'], "2016-01-01", "2023-03-01")

2799
MSFT (2520, 20)
           ticker        open        high         low       close   adj_close  \
date                                                                            
2016-01-04   MSFT   54.320000   54.799999   53.389999   54.799999   49.007942   
2016-01-05   MSFT   54.930000   55.389999   54.540001   55.049999   49.231529   
2016-01-06   MSFT   54.320000   54.400002   53.639999   54.049999   48.337219   
2016-01-07   MSFT   52.700001   53.490002   52.070000   52.169998   46.655922   
2016-01-08   MSFT   52.369999   53.279999   52.150002   52.330002   46.799007   
...           ...         ...         ...         ...         ...         ...   
2023-02-23   MSFT  255.559998  256.839996  250.479996  254.770004  254.770004   
2023-02-24   MSFT  249.960007  251.000000  248.100006  249.220001  249.220001   
2023-02-27   MSFT  252.460007  252.820007  249.389999  250.160004  250.160004   
2023-02-28   MSFT  249.070007  251.490005  248.729996  249.419998  249.419998   
2023-03

In [ ]:
spark.stop()